In [1]:
import pandas as pd

df = pd.read_csv('response.csv') 
df = df.fillna('')
df['response']=df.iloc[:,3]+df.iloc[:,5]+df.iloc[:,6]
df['issue'] = df.iloc[:,1]
df['symptom'] = df.iloc[:,2] + df.iloc[:,4]
subset = df.iloc[:,-3:]

In [2]:
import markovify
import codecs

model = markovify.NewlineText(subset.response, retain_original=False, state_size=3)

for i in range(10):
    print(model.make_short_sentence(200))

It makes a grinding sound for a few seconds and then nothing happens. 
Dashboard computer reboots when traveling above 80mph
When I drive, I feel like the car is making a clicking noise.
Shuttering when accelerating 
When I'm taking a left turn, the steering wheel shudders and I hear a grinding noise too.
The pedal feels squishy. It stops if I push the key in the ignition, I just get a clicking sound - maybe a dead battery?
windshield wipers 
When I turn the radio and asking me if I know where the All Spark is hidden
It makes clunking noises when I go over a bump.
I tried starting my car but it doesn't work. It sputters for a while and never turns over.


In [9]:
def train_markov_type(data, issue):
    return markovify.Text(data[data["issue"] == issue].response, retain_original=False, state_size=3)

def make_sentence(model, length=200):
    return model.make_short_sentence(length)

other_model = train_markov_type(subset, "Other")
brakes_model = train_markov_type(subset, "Brakes")
starter_model = train_markov_type(subset, "Starter")

In [10]:
make_sentence(other_model)

'The steering wheel shakes uncontrollably when I accelerate above 60 mph.'

In [11]:
make_sentence(brakes_model)

"It's rather a periodic deep sound when breaking."

In [12]:
make_sentence(starter_model)

'My car does not turn over'

We can combine these models with relative weights

In [13]:
compound_model = markovify.combine([other_model, brakes_model, starter_model], [14, 3, 3])

In [14]:
for i in range(20):
    print(make_sentence(compound_model))

The oil change light comes on and the fan is running full blast.
The steering wheel shakes uncontrollably when I accelerate above 60 mph.
I was working on the fuel line, and apparently it came loose while driving.
It stops if I push the gas peddle.
I'm on the side of the road, the lights, radio, electronic door locks, etc are all working fine, but the car won't shift out of park.
I can hear the starter firing but the car won't shift out of park.
The car pulls to the left.
The driver side window auto function does not work, you pull the window up, it goes all the way up and then down.
When turning, the car is leaning on left
the privacy glass between my driver and my seating area doesn't work well.
my car does not stop
When turning, the car is leaning on left
I don't know what more you want from me, I get into the car, turn the key, nothing happens.
the flux capacitor is misfiring and I can't get up to 88 MPH
When I drive, I feel like the car is making a clicking noise.
Not so much a gr

In [22]:
import numpy
def generate_cases(models, weights=None):
    if weights is None:
        weights = [1] * len(models)
    
    choices = []
    
    total_weight = float(sum(weights))
    
    for i in range(len(weights)):
        choices.append((float(sum(weights[0:i+1])) / total_weight, models[i]))
    
    def choose_model():
        r = numpy.random.uniform()
        for (p, m) in choices:
            if r <= p:
                return m
        return choices[-1][1]


    def choose_from(c):
        idx = math.floor(numpy.random.uniform() * len(c))
        return c[idx]
    
    
    while True:
        yield (make_sentence(choose_model()))
            

In [23]:
t = generate_cases([other_model, brakes_model, starter_model], [5,20,20])

In [24]:
[next(t) for i in range(100)]

["The rotors are warped so it's difficult to push the break pedal without lurching.",
 'It stops if I push it really far, but I have to brake again.',
 'If I pump the brake it works better.',
 'It makes a sputtering noise and does not turn over',
 'Have to cycle the ignition three times before the car starts on the the fourth time.',
 "Starter makes clicking noise, won't start the engine unless you hit the starter with a hammer.",
 'The car make some screeching or grinding noises when stopping.',
 'You must act immediately!',
 'While driving I hear lot of noises coming from the brakes of my car.',
 'While driving I hear lot of noises coming from the brakes of my car.',
 'The car make some screeching or grinding noises when stopping.',
 'brake pads are not wearing down evenly',
 'With that, highway driving is very dangerous',
 "Where's the place, where I need to insert my key to start the engine unless you hit the starter with a hammer.",
 'Sometimes when I brake, after a few seconds, t

In [25]:
import cProfile

def timing(c):
    for _ in range(c):
        next(t)

cProfile.run('timing(2000)', 'generatestats')

In [26]:
import pstats
p = pstats.Stats('generatestats')
p.strip_dirs().sort_stats(-1).print_stats()

Mon Mar  8 11:22:59 2021    generatestats

         282666 function calls (257990 primitive calls) in 0.140 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     2000    0.002    0.000    0.008    0.000 <ipython-input-22-5f1382f3447f>:13(choose_model)
     2000    0.001    0.000    0.139    0.000 <ipython-input-22-5f1382f3447f>:2(generate_cases)
        1    0.001    0.001    0.140    0.140 <ipython-input-25-801ca619797e>:3(timing)
     2000    0.001    0.000    0.129    0.000 <ipython-input-9-8ddb86548959>:4(make_sentence)
        1    0.000    0.000    0.140    0.140 <string>:1(<module>)
    55417    0.014    0.000    0.020    0.000 chain.py:10(accumulate)
    26704    0.017    0.000    0.110    0.000 chain.py:112(gen)
     2028    0.005    0.000    0.115    0.000 chain.py:125(walk)
    26704    0.061    0.000    0.093    0.000 chain.py:96(move)
     2028    0.001    0.000    0.002    0.000 text.py:120(word_join)
     2028